In [1]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_temperature_pressure import*
CP_param_file = os.path.join(root_path,'iltransr/pre-trained params/CP_best.params')

[04:44:48] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for CPU
[04:44:49] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for GPU


In [2]:
dropout = 0
batch_size = 256

In [ ]:
net = ILNet(dropout=dropout)
net.IL_encoder = model.encoder
net.IL_src_embed =  model.src_embed
net.hybridize()

In [4]:
net.load_parameters(CP_param_file,ctx=ctx)
net.initialize(init=mx.init.Xavier(magnitude=magnitude), ctx=ctx)

In [5]:
import pandas as pd
to_predict_database = pd.read_excel('smiles.xlsx',sheet_name='To_be_calculated')

In [6]:
predict_IL_smiles = to_predict_database['SMILES'].map(canonical_smile).map(no_split).map(preprocess)
predict_T =to_predict_database['normalized_T_CP']
predict_P = to_predict_database['normalized_P_CP']
predict_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(predict_IL_smiles,predict_T,predict_P))
predict_dataloader = get_predict_dataloader(predict_dataset, batch_size=batch_size)

In [7]:
predicted = predict(net, predict_dataloader,ctx)

[04:44:56] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable
[04:44:59] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable


In [8]:
df = pd.DataFrame(predicted)
df.to_csv('predicted_CP.csv')
print(df)

             0
0     5.994577
1     6.384035
2     6.315397
3     6.039165
4     5.986280
...        ...
5513  5.946232
5514  5.613517
5515  6.587558
5516  5.584871
5517  6.066499

[5518 rows x 1 columns]
